In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Krish Naik is a YouTuber and educator known for his work in the field of data science and machine learning. He creates educational content on platforms like YouTube, where he shares tutorials, courses, and insights related to data science, artificial intelligence, and programming. His videos often cover topics such as Python programming, machine learning algorithms, deep learning, and data visualization. Krish Naik is recognized for his engaging teaching style and practical approach to complex subjects, making them accessible to a wide audience.",
        metadata={
            "source": "https://www.youtube.com/@KrishNaik",
            "title": "Krish Naik",
        }),
    Document(
        page_content="I like Asian Baddies",
        metadata={
            "source": "My Brain",
            "title": "My preference",
        }),
    Document(
        page_content="I am talking to a few girls right now",
        metadata={
            "source": "My Whatsapp chats",
            "title": "Whatsapp"
        }),
    Document(
        page_content="Why am I like this? I don't know. The past has moulded me into this person. I am a product of my past experiences and choices.",
        metadata={
            "source": "My life experiences",
            "title": "My Life"})
]

In [5]:
documents

[Document(metadata={'source': 'https://www.youtube.com/@KrishNaik', 'title': 'Krish Naik'}, page_content='Krish Naik is a YouTuber and educator known for his work in the field of data science and machine learning. He creates educational content on platforms like YouTube, where he shares tutorials, courses, and insights related to data science, artificial intelligence, and programming. His videos often cover topics such as Python programming, machine learning algorithms, deep learning, and data visualization. Krish Naik is recognized for his engaging teaching style and practical approach to complex subjects, making them accessible to a wide audience.'),
 Document(metadata={'source': 'My Brain', 'title': 'My preference'}, page_content='I like Asian Baddies'),
 Document(metadata={'source': 'My Whatsapp chats', 'title': 'Whatsapp'}, page_content='I am talking to a few girls right now'),
 Document(metadata={'source': 'My life experiences', 'title': 'My Life'}, page_content="Why am I like th